In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from random import randint

In [ ]:
def ColorComparing(blue1, green1, red1, blue2, green2, red2):
    ColorComparing = sqrt( ((blue2 - blue1)**2) + ((green2- green1)**2) + ((red2 - red1)**2) ) 
    return ColorComparing

In [2]:
def Vregions(image, CentersOfClusters):
    V_regions = []
    for i in range(len(CentersOfClusters)): #Use the center pixel as the first element in each V_Region array
        V_region = []
        V_region.append(CentersOfClusters[i])
        V_regions.append(V_region)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]): 
            OptimalPoint = []   #Find Optimial point by comparing min and max Color
            MinimumColor = float('inf')
            for V_Cluster in CentersOfClusters:
                blue1, green1, red1 = image[i, j]
                blue2,green2,red2= V_Cluster['RGB']
                ColorRank = ColorComparing(blue1, green1, red1, blue2, green2, red2) #Get selected color
                if(ColorRank < MinimumColor):
                    MinimumColor = ColorRank
                    OptimalPoint = V_Cluster
            index = 0
            for V_region in V_regions: #find optimal point for cluster
                if(V_region[0]['idx'] == OptimalPoint['idx']):
                    blue,green,red = image[i, j]
                    V_regions[index].append({
                        'RGB': (blue, green, red),
                        'idx': (i,j)
                    })
                index += 1
    return V_regions

In [3]:
def KMeansAlgorithm(image, NumberOfClusters):
    CentersOfClusters = []
    Threshold = 30
    for i in range(NumberOfClusters):
        KMeansCluster = {}
#Use random Values    
        # def RandomPoints(image,k):
#     RandomPoint = []
#     c_RandomPoint = []
#     for i in range(k):
#         r = randint(0, image.shape[0])
#         c = randint(0, image.shape[1])
#         RandomPoint.append((r, c))
#     for i in RandomPoint:
#         blue,green,red = image[i]
#         c_RandomPoint.append((blue,green,red))
#     return c_RandomPoint
        r = randint(0, image.shape[0] - 1) 
        c = randint(0, image.shape[1] - 1)
        KMeansCluster['idx'] = (r, c)
        blue,green,red = image[r, c]
        KMeansCluster['RGB'] = (blue, green, red)
        CentersOfClusters.append(KMeansCluster)
    IsImageConverged = False #finising populating center of clusters
    while not IsImageConverged:
        KMeansRegions = Vregions(image, CentersOfClusters) #Outline V_Regions, then analyze
        KMeansRegionsConverge = 0
        KMeansColor = []
        for KMeanRegion in KMeansRegions:
            NumPixels = 0
            blue = 0
            green = 0 
            red = 0
            for PixelPostion in KMeanRegion: 
                blue += PixelPostion['RGB'][0]
                green += PixelPostion['RGB'][1]
                red += PixelPostion['RGB'][2]
                NumPixels += 1
            AvgBlue = blue // NumPixels
            AvgGreen = green // NumPixels
            AvgRed = red // NumPixels
            KMeansColor.append((AvgBlue, AvgGreen, AvgRed))
        for i in range(len(KMeansRegions)): #Converge the Regions
            blue1,green1,red1 = KMeansColor[i]
            blue2,green2,red2 = KMeansRegions[i][0]['RGB']
            SimOfColor = ColorComparing(blue1, green1, red1, blue2, green2, red2)
            if(SimOfColor < Threshold):
                KMeansRegionsConverge += 1
        if(KMeansRegionsConverge == len(KMeansRegions)): #Finish Converging
            for i in range(len(KMeansRegions)):
                for PixelPostion in KMeansRegions[i]:
                    image[PixelPostion['idx']] = KMeansColor[i]
            IsImageConverged = True
        else: #Could not converge
            for i in range(len(KMeansRegions)):
                image[KMeansRegions[i][0]['idx']] = KMeansColor[i]
                CentersOfClusters[i]['RGB'] = KMeansColor[i]
    return image

In [4]:
DisplayMessage = "KMeans Clustering to Segment an Image"
def KMeansClustering(image): 
    temp_img = image.copy()
    def KAlgorithm(k): 
        clustered = KMeansAlgorithm(temp_img, k)
    def KMeansTrackbar():
        cv2.namedWindow(DisplayMessage)
        cv2.createTrackbar("K-Value", DisplayMessage, 0, 25, KAlgorithm)
        cv2.setTrackbarPos("K-Value", DisplayMessage, 15) #Change starting trackbar value here
        cv2.imshow(DisplayMessage, temp_img)   
        cv2.waitKey(0)
        cv2.destroyAllWindows()    
    KMeansTrackbar()

In [5]:
BaseImage = cv2.imread('baboon.jpg')
KMeansClustering(BaseImage)

<ipython-input-1-8fd1e948fec6>:8: RuntimeWarning: overflow encountered in ubyte_scalars
  ColorComparing = sqrt( ((blue2 - blue1)**2) + ((green2- green1)**2) + ((red2 - red1)**2) )


In [5]:
BaseImage = cv2.imread('SAFlag.jpg')
KMeansClustering(BaseImage)

<ipython-input-1-05c0f814c830>:20: RuntimeWarning: overflow encountered in ubyte_scalars
  ColorComparing = sqrt( ((blue2 - blue1)**2) + ((green2- green1)**2) + ((red2 - red1)**2) )
